In [1]:
### 중요: 이 코드는 인과적 어텐션 메커니즘을 구현한 PyTorch 모듈입니다.
import torch
import torch.nn as nn

class CausalAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, qkv_bias=False):
        """
        Args:
            d_in: 입력 벡터의 차원 크기
            d_out: 출력(및 쿼리/키/밸류) 벡터의 차원 크기
            context_length: 모델이 한 번에 처리할 수 있는 최대 문맥 길이 (토큰 수)
            dropout: 드롭아웃 확률
            qkv_bias: 선형 레이어에 편향(bias)을 사용할지 여부
        """
        super().__init__()
        
        # 1. 쿼리(Query), 키(Key), 밸류(Value)를 만들기 위한 선형 투영 레이어 정의
        # 입력 벡터(d_in)를 각각의 목적에 맞는 벡터(d_out)로 변환합니다.
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

        # 2. 과적합(Overfitting) 방지를 위한 드롭아웃 설정
        self.dropout = nn.Dropout(dropout)

        # 3. 인과적 마스크(Causal Mask) 생성 및 버퍼 등록
        # 'register_buffer'를 사용하면 역전파(학습) 대상은 아니지만, 모델의 상태(state_dict)로 저장됩니다.
        # torch.triu(..., diagonal=1): 대각선(0) 위쪽 삼각형 부분만 1로 채웁니다.
        # 즉, '미래의 정보' 위치에 1을 표시하여 나중에 가릴(masking) 준비를 합니다.
        self.register_buffer(
            'mask', 
            torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )

    def forward(self, x):
        # x.shape: [배치 크기(b), 토큰 개수(num_tokens), 입력 차원(d_in)]
        b, num_tokens, d_in = x.shape 

        # 입력 x를 통과시켜 현재 시점의 관심사(Query), 검색 대상(Key), 정보 내용(Value)을 추출합니다.
        # region [Q, K, V 벡터 계산]
        keys = self.W_key(x)        # Shape: [b, num_tokens, d_out]
        queries = self.W_query(x)   # Shape: [b, num_tokens, d_out]
        values = self.W_value(x)    # Shape: [b, num_tokens, d_out]
        # endregion

        # Query와 Key의 내적(Dot Product)을 통해 각 토큰 간의 관련성을 구합니다.
        # keys.transpose(1, 2): 행렬 곱을 위해 차원을 뒤집습니다. (d_out 차원끼리 곱해짐)
        # region [어텐션 스코어(유사도) 계산]
        attn_scores = queries @ keys.transpose(1, 2) 
        #endregion

        # mask가 1인 위치(미래 시점의 토큰들)를 -무한대(-inf)로 채웁니다.
        # 이렇게 하면 나중에 Softmax를 거칠 때 확률이 0이 되어, 미래 정보를 참조하지 못하게 됩니다.
        # [:num_tokens, :num_tokens]: 입력 길이가 context_length보다 짧을 때를 대비해 크기를 맞춥니다.
        # region [인과적 마스킹 (Masking) - 미래 정보 차단]
        attn_scores.masked_fill_(
            self.mask.bool()[:num_tokens, :num_tokens], 
            -torch.inf
        ) 
        # endregion

        # 스케일링(/ keys.shape[-1]**0.5): 차원이 커질수록 내적 값이 커져 기울기 소실이 오는 것을 방지합니다.
        # Softmax: 점수를 확률(0~1 사이, 합은 1)로 변환합니다.
        # region [어텐션 가중치(Weights) 계산 및 스케일링]
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        # endregion

        # 계산된 가중치 중 일부를 무작위로 0으로 만들어 모델이 특정 토큰에만 의존하는 것을 막습니다.
        # region [드롭아웃 적용]
        attn_weights = self.dropout(attn_weights)
        # endregion

        # 어텐션 가중치(확률)를 기반으로 Value(정보)들을 가중 합산합니다.
        # 결과적으로 "현재 토큰과 관련이 깊은 과거 토큰들의 정보"가 진하게 섞인 벡터가 됩니다.
        # region [문맥 벡터(Context Vector) 생성]
        context_vec = attn_weights @ values 
        # endregion
        
        return context_vec



In [2]:
import torch

import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

d_in = inputs.shape[1]   # 입력 차원 (d=3)
d_out = 2                # Q, K, V의 출력 차원 (d=2)

batch = torch.stack((inputs, inputs), dim=0)
# --- 실행 예시 ---
torch.manual_seed(123)

# 가정: batch 변수가 이미 정의되어 있다고 가정 (예: b=2, num_tokens=6, d_in=...)
# context_length는 모델이 허용하는 최대 길이이므로, 현재 배치의 길이와 같거나 더 길게 설정합니다.
context_length = batch.shape[1] 

ca = CausalAttention(d_in, d_out, context_length, 0.0)
context_vecs = ca(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]],

        [[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]]], grad_fn=<UnsafeViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])
